In [15]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import jieba
import re

In [17]:
with open("./weibos.txt","r",encoding="utf-8")as f:
    text=f.read()

In [10]:
text=text.replace("\u200b","").replace("\n","").replace(" ","")
sentences=re.split('[。！？]', text)

In [11]:
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [13]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [32]:
# 设置停用词
with open('cn_stopwords.txt',"r",encoding="utf-8")as f:
    stop = [line.strip() for line in f.readlines()]
    
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

In [33]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [37]:
query = '斯科拉里愿意执教国足'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)


8 0.1484375 意味着本来没打算球队管理层中国足协高层赛后第一时间进行辞职对话
27 0.4140625 中国队斯科拉里开足薪水足需要条件
30 0.3515625 支持斯科拉里
Top 3 邻居 [8, 27, 30]
